# KLUE-TC(YNAT) dataset (2800개)

- Labeling Error - 1000개는 라벨을 임의로 바꿈
→ 올바른 라벨로 재맵핑 필요
- Random Noise - 1600개는 text에 노이즈를 추가. 임의의 char 중 20~80%를 랜덤으로 다른 아스키코드로 대체
→ 높은 노이즈를 가지는 text는 제외, 낮은 샘플은 복구가 필요
→ 사람이 개별 판단하지 말 것. 자동 detection이 가능하게 하여 수정하거나, 삭제하거나 진행
- Normal Dataset - 200개

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import os
from rank_bm25 import BM25Okapi
from konlpy.tag import Okt

In [ ]:
df = pd.read_csv(os.path.join(os.getcwd(), 'data/train.csv'))
df.head()

In [ ]:
# 화살표가 포함된 행을 필터링
arrows = ['→', '←', '↑', '↓', '↔']
mask = df['text'].str.contains('|'.join(arrows))  # 화살표 기호가 포함된 행 찾기
df_with_arrows = df[mask]

# 결과 출력
df_with_arrows.head()

In [ ]:
# 공백으로 대체
df['text'] = df['text'].str.replace('…', ' ', regex=False)
df['text'] = df['text'].str.replace('...', ' ', regex=False)
df['text'] = df['text'].str.replace('·', ' ', regex=False)
# 화살표 기호 의미 대체
df['text'] = df['text'].str.replace('→', '에서', regex=False)
df['text'] = df['text'].str.replace('↑', '상승', regex=False)
df['text'] = df['text'].str.replace('↓', '하락', regex=False)
df['text'] = df['text'].str.replace('↔', ' ', regex=False)

In [ ]:
# 정규 표현식 정의: 소수점 뒤에 숫자가 있으며, 그 뒤에 '%'가 있는 패턴
pattern = r'\d+\.\d+%'

# '.숫자%' 구성으로 이루어진 데이터 필터링
df_filtered = df[df['text'].str.contains(pattern, regex=True)]

# 결과 출력
df_filtered.head()

In [ ]:
# 특수 기호의 패턴 정의
# 이 예시에서는 영숫자 이외의 모든 기호를 특수 기호로 간주합니다.
# special_char_pattern = r'[^가-힣A-Z\u4E00-\u9FFF\s0-9]'
# .숫자% 형식을 제외하고, ㎜도 특수 기호에서 제외
special_char_pattern = r'(?<!\d)\.(?!\d)|(?<!\d)%|[^가-힣A-Z\u4E00-\u9FFF\s0-9\.%㎜㎡]'

# 각 text에 포함된 특수 기호의 개수를 세는 함수 정의
def count_special_characters(text):
    return len(re.findall(special_char_pattern, text))

# 데이터프레임에 새로운 열 추가
df['special_char_count'] = df['text'].apply(count_special_characters)
df['special_char_ratio'] = df['special_char_count'] / df['text'].str.len()

df_sorted = df.sort_values(by='special_char_ratio', ascending=False)

# 결과 출력
df_sorted.head()

In [ ]:
# 출력 옵션을 설정하여 모든 행이 표시되도록 함
pd.set_option('display.max_rows', None)

# special_char_ratio가 0.2 이상인 데이터 필터링
noise_df = df_sorted[df_sorted['special_char_ratio'] >= 0.042]
print(len(noise_df))

# 결과 출력
noise_df.head()

# 출력 옵션을 원래대로 복구
pd.reset_option('display.max_rows')

In [ ]:
# df_high_ratio의 인덱스를 이용하여 제외할 행을 찾음
clean_df = df[~df.index.isin(noise_df.index)]

print(len(clean_df))
clean_df.head()

# 클러스터링 기반 라벨링 확인

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(clean_df['text'])
X

In [ ]:
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X.toarray())

In [ ]:
kmeans = KMeans(n_clusters=7, random_state=42)
clean_df['cluster'] = kmeans.fit_predict(X_pca)

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=clean_df['target'], palette='Set1', s=100, alpha=0.6, edgecolor='w')
plt.title('PCA로 축소된 뉴스 제목의 클러스터링 결과')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend(title='Actual Label')
plt.show()

In [ ]:
clean_df['label_match'] = clean_df['target'] == clean_df['cluster']
mismatched_labels = clean_df[clean_df['label_match'] == False]
print(f"잘못 라벨링된 데이터 수: {len(mismatched_labels)}")


In [ ]:
label_map = {
    0:'생활문화',1:'스포츠',2:'정치',3:'사회',4:'IT과학',5:'경제',6:'세계'
}
mismatched_labels['target'] = mismatched_labels['target'].map(label_map)
mismatched_labels['cluster'] = mismatched_labels['cluster'].map(label_map)
mismatched_labels.head()

직접 비교해본 결과, 정확도가 떨어진다. 1000개 데이터 이상치를 발견한 것은 우연인 듯하다.
# BM25 활용

In [ ]:
okt = Okt()